In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('..\\data\\processed2\\final_dataset.csv', index_col=0)
df = df[df['batter event flag']=='T']
df.reset_index(0, inplace=True, drop=True)

In [3]:
batter_event = []
modifier = []
advances = []
for event in df['event text']:
    s = event.split('.')
    if len(s) > 1:
        ev = s[0].split('/')
        if len(ev) > 1:
            batter_event += [ev[0]]
            modifier += [ev[1]]
        else:
            batter_event += ev
            modifier += ['']
        advances += [s[-1]]
    else:
        ev = s[0].split('/')
        if len(ev) > 1:
            batter_event += [ev[0]]
            modifier += [ev[1]]
        else:
            batter_event += ev
            modifier += ['']
        advances += ['']
df['batter_event'] = batter_event
df['modifier'] = modifier
df['advances'] = advances

In [4]:
games = df.gameid.unique()
batters = df['res batter'].unique()
pitchers = df['res pitcher'].unique()

In [5]:
df['batter_event'] = [e.replace('!','') for e in df['batter_event']]

In [6]:
replace_chars = ['+', '*', '.', '1', '2', '3', '>']
for char in replace_chars:
    df['pitch sequence'] = [s.replace(char, '') for s in df['pitch sequence']]

In [7]:
df['num_pitches'] = [len(s) for s in df['pitch sequence']]
df['strikeout looking'] = [s[-1]=='C' for s in df['pitch sequence']]

In [8]:
df['fouls'] = df['num_pitches'] - df['strikes'] - df['balls']

In [9]:
df['event_final'] = df['batter_event']

In [10]:
df.loc[[i for i in df.index if 'IW' in df.loc[i, 'batter_event']], 'event_final'] = 'IW'

In [11]:
df.loc[df['strikeout looking']==True, 'event_final'] = 'C'

In [12]:
df.loc[[i for i in df.index if df.loc[i, 'batter_event'][:1] == 'W'], 'event_final'] = 'W'

In [13]:
df['modifier'] = [m.replace('-', '') for m in df['modifier']]
df['modifier'] = [m.replace('+', '') for m in df['modifier']]

In [14]:
mod_map = {'' : '',
           '7' : '7',
           '8' : '8',
           '9' : '9',
           'F' : 'F',
           'L' : 'L',
           'P' : 'P',
           'B' : 'B',
           'AP' : 'G',
           'TP' : 'G',
           'DP' : 'G',
           'FO' : 'G',
           'G' : 'G',
           'TH' : 'G',
           'IF' : 'P',
           'SF' : 'SF',
           'SH' : 'SH',
           '78' : '78',
           '89' : '89',}
def mod_mapper(mod_series, mod_map):
    mod_out = []
    for m in mod_series:
        out = None
        for key in mod_map.keys():
            if m[:len(key)] == key:
                out = mod_map[key]
        if out == None:
            print(m)
        else:
            mod_out += [out]
    return mod_out

In [15]:
modifier_trim = mod_mapper(df['modifier'], mod_map)

In [16]:
df['modifier simple'] = modifier_trim

In [17]:
ev = []
for s in df['batter_event']:
    if ('S' in s) or ('D' in s) or ('T' in s) or ('E' in s):
        ev += [s[:2]]
    elif ('K' in s):
        ev += [s[:2]]
    else:
        ev += [s[0]]

In [18]:
first = [0]*len(df)
second = [0]*len(df)
third = [0]*len(df)
for i in df.index:
    if isinstance(df.loc[i, '1st runner'], str):
        first[i] = 1
    if isinstance(df.loc[i, '2nd runner'], str):
        second[i] = 1
    if isinstance(df.loc[i, '3rd runner'], str):
        third[i] = 1
    if i % 1000 == 0:
        print(i)
first = np.array(first)
second = np.array(second)
third = np.array(third)
df['first'] = first
df['second'] = second
df['third'] = third

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [19]:
event_final = [e for e in df['event_final']]
for i in df.index:
    if ev[i][0].isnumeric():
        event_final[i] = ev[i]+modifier_trim[i]
    elif (ev[i][0]=='S') or (ev[i][0]=='D') or (ev[i][0]=='T'):
        event_final[i] = ev[i]+modifier_trim[i]
    elif ev[i][0]=='H':
        event_final[i] = ev[i]+modifier_trim[i]
    elif ev[i][0]=='K':
        if len(ev[i]) > 1:
            event_final[i] = 'K'
    elif ev[i][0]=='E':
        event_final[i] = ev[i][1]+modifier_trim[i]
    if i % 1000 == 0:
        print(i)
df['event_final'] = event_final


0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [20]:
df.loc[df['event_final']=='1EB', 'event_final'] = '1B'

In [21]:
df['event_final'].value_counts()[60:120]

9SF     1872
T8L     1851
5B      1790
S1B     1731
3B      1724
7SF     1701
T8F     1556
FC6     1317
5SH     1314
DG7     1283
3SH     1282
DG9     1273
FC1     1234
T9F     1170
FC5     1152
2SH     1109
S1L      902
DG8      765
FC3      739
S4L      728
S6L      715
S4P      616
DG78     584
T7L      583
DG89     576
S3B      562
FC4      556
1P       552
S6P      534
T7F      473
T9G      471
S2B      456
D5G      452
S5L      439
D8G      355
S2G      333
S3L      333
S4B      285
3EG      237
D3G      190
S5P      170
DGL      166
D4P      162
S3P      160
FC2      160
D6P      156
S6B      147
D6G      147
4B       139
9G       137
6EG      134
DGF      131
T7G      128
D5L      114
5EG      107
4EG       97
D4G       89
HF        85
8G        67
1EG       67
Name: event_final, dtype: int64

In [22]:
other_cats = df['event_final'].unique()[df['event_final'].value_counts() < 130]

In [23]:
event_final = [e for e in df['event_final']]
for i in df.index:
    if event_final[i] in other_cats:
        event_final[i] = 'other'

In [24]:
df['event_final'] = event_final

In [25]:
df['event_final'].value_counts()[60:]

S5B     1924
9SF     1872
T8L     1851
5B      1790
S1B     1731
3B      1724
7SF     1701
T8F     1556
FC6     1317
5SH     1314
DG7     1283
3SH     1282
DG9     1273
FC1     1234
T9F     1170
FC5     1152
2SH     1109
S1L      902
DG8      765
FC3      739
S4L      728
S6L      715
S4P      616
T7L      583
DG89     576
S3B      562
FC4      556
1P       552
S6P      534
T7F      473
T9G      471
S2B      456
D5G      452
S5L      439
D8G      355
S2G      333
S3L      333
S4B      285
3EG      237
D3G      190
S3P      160
FC2      160
S6B      147
4B       139
9G       137
T7G      128
D4G       89
HF        85
SB        53
D1G       34
DG        32
6SF        9
34G        3
Name: event_final, dtype: int64

In [26]:
df.to_csv('..\\data\\processed2\\processed_data2.csv')